In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# #Install Apache Spark 3.2.1 with Hadoop 3.2, get zipped folder
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz    

# #Unzip folder
!tar xf spark-3.3.1-bin-hadoop3.tgz

# #Install findspark, pyspark 3.2.1
!pip install -q findspark
!pip install pyspark==3.2.1

# #Set variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853643 sha256=ebf9da5a62eb1dcd8fd4a7213c5a5763d6d62b340d1a2fd6d3f1f78c11e45832
  Stored in directory: /root/.cache/pip/wheels/58/94/83/915c9059e4b038e2d43a6058f307fe1c3e8536e5745f3b23b7
Successfully built pyspark


In [ ]:
pip install elasticsearch==7.17.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 385 kB 4.5 MB/s 


In [ ]:
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
from IPython.display import Image, HTML, display
import elasticsearch
from elasticsearch import Elasticsearch
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import col, mean, udf, lit, current_timestamp, unix_timestamp, array_contains
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

schemaMetaData = StructType([
    StructField("_id", StringType(), True), \
    StructField("asin", StringType(), False), \
    StructField("similar", StringType(), False)])

schemaReviewData = StructType([
    StructField("_id", StringType(), True), \
    StructField("asin", StringType(), False), \
    StructField("reviewerID", StringType(), False), \
    StructField("reviewerName", StringType(), False), \
    StructField("overall", FloatType(), False), \
    StructField("verified", StringType(), False)])


In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)
spark.conf.set("spark.sql.caseSensitive", "true")
spark

In [ ]:
dfMetaData = spark.read.json("/content/metaVinylAndCD.txt", schema=schemaMetaData)
dfReviewData = spark.read.json("/content/VinylAndCD.txt", schema=schemaReviewData)
dfReviewData = dfReviewData.withColumn("verified",dfReviewData["verified"].cast(BooleanType()))

In [ ]:
#start the engine first

# test your ES instance is running
from elasticsearch import Elasticsearch

es = Elasticsearch(
    ['s-for-ds.es.us-central1.gcp.cloud.es.io'],
    http_auth=('elastic', 'yRG2A89HY5IbVuBinEtu8F8x!'),
    scheme="https",
    port=9243
)

In [ ]:
# test block
from google.colab import drive
drive.mount('/content/drive')

! python -m pip install pymongo==3.7.2

import pymongo
from pymongo import MongoClient

# uri (uniform resource identifier) defines the connection parameters 
uri = "<your string here>"
# start client to connect to MongoDB server 
client = MongoClient( uri )

client.list_database_names()

dfMetaData = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/metaData.VinylAndCD").load()
dfReviewData = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/metaData.VinylAndCD").load()

In [ ]:
# Data Processing starts here:

In [ ]:
dfMetaData.show()

+--------------------+----------+----------+
|                 _id|      asin|   similar|
+--------------------+----------+----------+
|638cfdc61ca6f99a8...|0001393774|B0016CP2GS|
|638cfdc61ca6f99a8...|0001393774|1423442733|
|638cfdc61ca6f99a8...|0001393774|B00P826ZRM|
|638cfdc61ca6f99a8...|0001393774|B000075A86|
|638cfdc61ca6f99a8...|0001393774|0006920055|
|638cfdc61ca6f99a8...|0001501348|6303646611|
|638cfdc61ca6f99a8...|0001501348|B000008GO1|
|638cfdc61ca6f99a8...|0001501348|6303908845|
|638cfdc61ca6f99a8...|0001501348|B00000DUDE|
|638cfdc61ca6f99a8...|0005072298|B00000DPLJ|
|638cfdc61ca6f99a8...|0005072298|B00KYVHGR0|
|638cfdc61ca6f99a8...|0005072298|B00BQH9RKA|
|638cfdc61ca6f99a8...|0005072298|B00000DPLL|
|638cfdc61ca6f99a8...|0005072298|B000008UPG|
|638cfdc61ca6f99a8...|0005121515|B00000DPKH|
|638cfdc61ca6f99a8...|0005121515|B0010WAC50|
|638cfdc61ca6f99a8...|0005121515|B0002X7GAM|
|638cfdc61ca6f99a8...|0005121515|B00FSZPM8C|
|638cfdc61ca6f99a8...|0005121515|0005121450|
|638cfdc61

In [ ]:
dfReviewData.show()

+--------------------+----------+--------------+----------------+-------+--------+
|                 _id|      asin|    reviewerID|    reviewerName|overall|verified|
+--------------------+----------+--------------+----------------+-------+--------+
|638cfe401ca6f99a8...|0001393774|A171I27YBM4FL6|       ElineMann|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A1H1DL4K669VQ9|  JudithPaladino|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A23WIHT5886G36| MarkLeszczynski|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A3SZNOJP8OL26X|            mimi|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A3V5XBBT7OZG5G|          gflady|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A3SNL7UJY7GWBI| LadyLeatherneck|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A3478QRKQDOPQ2|           jacki|    5.0|    true|
|638cfe401ca6f99a8...|0001393774|A3CP0CNKNFCYBZ|VirginiaCosgrove|    4.0|    true|
|638cfe401ca6f99a8...|0001393774|A3OIIDZ137NJOU|  PamelaT.Martin|    5.0|    true|
|638

In [ ]:
# Ratings for 
ratingsDF = dfReviewData.groupBy("asin").mean()
ratingsDF.printSchema()
ratingsDF.show()

root
 |-- asin: string (nullable = true)
 |-- avg(overall): double (nullable = true)

+----------+------------------+
|      asin|      avg(overall)|
+----------+------------------+
|6305214328|3.6666666666666665|
|B00000084B|               2.5|
|B000000DC7|               5.0|
|B000000D3O| 4.111111111111111|
|B000000EIH|               5.0|
|B000000HML|               5.0|
|B000000OSL| 4.566666666666666|
|B000000WI4|               4.0|
|B000000Y4Z|               4.8|
|B000000YLS| 4.916666666666667|
|B00000106O| 4.882352941176471|
|B0000015K2| 4.333333333333333|
|B000000148| 4.764705882352941|
|B000000DGC|               5.0|
|B000000P1I| 4.714285714285714|
|B000000W56|3.9130434782608696|
|B0000011P7| 4.166666666666667|
|B00000179Z|               5.0|
|5559550833| 4.786885245901639|
|6304749007| 4.416666666666667|
+----------+------------------+
only showing top 20 rows



In [ ]:
print("Total Unique Ratings: =>", ratingsDF.count())

Total Unique Ratings: => 2897


In [ ]:
# merging tables to access values
from pyspark.sql.functions import *

dup_cols = ["_id", "asin"]
mergedTable = dfMetaData.join(dfReviewData, ['asin']).where(dfMetaData.asin == dfReviewData.asin).drop("_id")
mergedTable.show()
print(mergedTable.count())

+----------+----------+--------------+--------------------+-------+--------+
|      asin|   similar|    reviewerID|        reviewerName|overall|verified|
+----------+----------+--------------+--------------------+-------+--------+
|0001393774|B0016CP2GS| AEKGGV851HY3K|          AvidReader|    5.0|   false|
|0001393774|B0016CP2GS|A1WX42M589VAMQ|                 Mir|    5.0|   false|
|0001393774|B0016CP2GS|A3IWBNNKB5Y1ZY|         Serendipity|    5.0|    true|
|0001393774|B0016CP2GS|A1JE8B5PU9ISMC|       MichelleToste|    5.0|    true|
|0001393774|B0016CP2GS| AWA4RHNCRLQNS|           S.Johnson|    5.0|   false|
|0001393774|B0016CP2GS|A12R54MKO17TW0|             J.Bynum|    5.0|   false|
|0001393774|B0016CP2GS|A2CNQ98AQMTF1T|       ShaunaLadnier|    5.0|    true|
|0001393774|B0016CP2GS|A1BA3VR9EKXI87|           M.Mikeska|    5.0|    true|
|0001393774|B0016CP2GS| AHG1GTQZUYNJN| CAROLYNECHAMBERLAIN|    5.0|    true|
|0001393774|B0016CP2GS|A1C7NPVPFF4OO8|        LisaL.Powell|    5.0|    true|

In [ ]:
ratingsDF = ratingsDF.withColumnRenamed("asin","prodID")
mergedTable2 = dfMetaData.join(ratingsDF, dfMetaData["similar"] == ratingsDF["prodID"], "inner")

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

windowAsin = Window.partitionBy("asin").orderBy(col("avg(overall)").desc())

fdf2 = mergedTable2.withColumn("row",row_number().over(windowAsin))

fdf2.select("similar", "avg(overall)").where(fdf2["asin"] == "B00000050L").show()

+----------+------------+
|   similar|avg(overall)|
+----------+------------+
|B00000052G|         5.0|
|B0000000ZW|        4.36|
+----------+------------+



In [ ]:
def getPrediction(product):
    ans = fdf2.select("similar", "avg(overall)").where(fdf2["asin"] == product)
    recommendationURLs = []

    for row in ans.select("similar").collect():
        recommendationURLs.append(f"https://www.amazon.com/gp/product/{row[0]}/r")

    return recommendationURLs

In [ ]:
import webbrowser, time
inputID = input()
a = time.time()
urls = [getPrediction(inputID)]
b = time.time()
print('Time Taken is: {}'.format(b-a))
for url in urls:
    print(url)

B00000050L
Time Taken is: 1.2554309368133545
['https://www.amazon.com/gp/product/B00000052G/r', 'https://www.amazon.com/gp/product/B0000000ZW/r']
